In [1]:
import torch
from torch import nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Types of Classifications
#    - Binary Classification (is this spam or not?)
#    - Multi Class Classification (is this a photo of sushi, steak or pizza?)
#    - Multi Label Classification (what tags should this article have?)
#    - https://www.learnpytorch.io/02_pytorch_classification/

In [ ]:
# Architecture of a Classification Neural Network

#    - Input Layer Shape (Same as number of features)
#    - Hidden Layers
#    - Neurons per Hidden Layer
#    - Output Layer Shape
#    - Hidden Layer Activation (usually ReLU)
#    - Output Activation (Sigmoid)
#    - Loss Function (Binary Crossentropy - torch.nn.BCELoss)
#    - Optimizer (SGD - Stochastic Gradient Descent)

In [2]:
# 1. Ready Classification Data

data = pd.read_csv('IRIS.csv') # import dataframe as pandas
#data.replace("Iris-setosa", 0, inplace=True)
#data.replace("Iris-versicolor", 1, inplace=True)
#data.replace("Iris-virginica", 2, inplace=True)

datanp = data.to_numpy() # convert pandas to numpy, shuffle data

x = np.zeros((150, 3), dtype=int)
datanp = np.concatenate([datanp, x], axis=1)

datanp[datanp[:, 4] == 'Iris-setosa', 5] = 1
datanp[datanp[:, 4] == 'Iris-versicolor', 6] = 1
datanp[datanp[:, 4] == 'Iris-virginica', 7] = 1

np.random.shuffle(datanp)

# Split into testing, training data & convert to PyTorch Tensors
trainX, trainY, testX, testY = datanp[0:140, 0:4].astype('float32'), datanp[0:140, 5:8].astype('float32'), datanp[140:, 0:4].astype('float32'), datanp[140:, 5:8].astype('float32')
trainX, trainY, testX, testY = torch.from_numpy(trainX), torch.from_numpy(trainY), torch.from_numpy(testX), torch.from_numpy(testY)



In [5]:
# 2. Build a Model


# 1. Set up device agnostic code (CPU or GPU if Available)
device = "cuda" if torch.cuda.is_available() else "cpu"

# 2. Construct Model using nn.model

class sepalClassifier(nn.Module): # Option 1: Use nn.Module Class
    def __init__(self):
        super().__init__()
        self.layer_1 = nn.Linear(in_features=4, out_features=8) 
        self.layer_2 = nn.Linear(in_features=8, out_features=8)
        self.layer_3 = nn.Linear(in_features=8, out_features=3)
        self.relu = nn.ReLU()
        # 4 -> 8 -> 8 -> 3

    def forward(self, x):
        return self.layer_3(self.relu(self.layer_2(self.relu(self.layer_1(x)))))

model_0 = sepalClassifier().to(device)
    
"""

model_0 = nn.Sequential( # Option 2: use nn.Sequential
    nn.Linear(in_features=4, out_features=6),
    nn.Linear(in_features=6, out_features=1)).to(device)
    
"""

# 3. Define Loss Function & Optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model_0.parameters(), lr=0.01)

In [12]:
print(torch.cuda.get_device_name(torch.cuda.current_device()))

NVIDIA GeForce RTX 3060 Laptop GPU


In [10]:
model_0.train()
y =model_0(trainX).squeeze()

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat1 in method wrapper_CUDA_addmm)

In [7]:
# 4. Train & Test Loop
epochs = 2000

for epoch in range(epochs):
    ### Training

    model_0.train()                    # Put model in training mode (this is the default state of a model)

    y_pred = model_0(trainX)           # 1. Forward pass on train data using the forward() method inside 
    loss = loss_fn(y_pred, trainY)     # 2. Calculate the loss (how different are our models predictions to the ground truth)    
    optimizer.zero_grad()              # 3. Zero grad (Reset) of the optimizer
    loss.backward()                    # 4. Loss backwards
    optimizer.step()                   # 5. Progress the optimizer

    
    
    ### Testing
    
    model_0.eval()                     # Put the model in evaluation mode

    with torch.inference_mode():
      test_pred = model_0(testX)                                  # 1. Forward pass on test data
      test_loss = loss_fn(test_pred, testY)     # 2. Caculate loss on test data

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat1 in method wrapper_CUDA_addmm)

In [7]:
loss_fn(test_pred, testY)

tensor(0.1057)

In [8]:
# Softmax - Prediction Probabilities
torch.softmax(test_pred, dim=1)
torch.round(torch.softmax(test_pred, dim=1), decimals = 3)

tensor([[0.0010, 0.9550, 0.0440],
        [0.9980, 0.0020, 0.0000],
        [0.9950, 0.0050, 0.0000],
        [0.0000, 0.3990, 0.6000],
        [0.9950, 0.0050, 0.0000],
        [0.9970, 0.0030, 0.0000],
        [0.9950, 0.0050, 0.0000],
        [0.0010, 0.9790, 0.0200],
        [0.0000, 0.3660, 0.6340],
        [0.0010, 0.9960, 0.0040]])

In [9]:
testY

tensor([[0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 1., 0.]])